In [185]:
import meshplot as mp

In [186]:
import numpy as np
import trimesh

from trimesh import proximity, sample

In [196]:
# Utils 

def barycentric_coords(tris, P):
    """
    Return the barycentric coordinates of a point with respect to a triangle.
    """
    v0 = tris[2] - tris[0]
    v1 = tris[1] - tris[0]
    v2 = P - tris[0]
    d00 = np.dot(v0, v0)
    d01 = np.dot(v0, v1)
    d11 = np.dot(v1, v1)
    d20 = np.dot(v2, v0)
    d21 = np.dot(v2, v1)
    denom = d00 * d11 - d01 * d01
    v = (d11 * d20 - d01 * d21) / denom
    w = (d00 * d21 - d01 * d20) / denom
    u = 1.0 - v - w
    return np.array([u, v, w])


In [242]:
mesh_name = 'monkey'
mesh = trimesh.load(f'./data/{mesh_name}.obj')
hair = trimesh.load(f'./data/{mesh_name}_fur.obj')

amount = 2000

# sample point cloud
# child_roots = mesh.sample_points_poisson_disk(amount, 1.5)
child_roots, faces = sample.sample_surface(mesh, amount)
child_normals = mesh.face_normals[faces]

unable to load materials from: monkey_fur.mtl


In [243]:
from scipy.spatial import cKDTree

strand_len = 5
strands = np.array(np.split(hair.vertices, hair.vertices.shape[0] // strand_len))

tree = cKDTree(strands[:,0])
parents_pos, parents_ids = tree.query(child_roots, k=3)

parent_roots = np.array(strands[parents_ids])

In [244]:
weights = [barycentric_coords(p[:, 0], c) for p, c in zip(parent_roots, child_roots)]

childStrands = np.array([b[0] * (t[0] - t[0][0]) 
                       + b[1] * (t[1] - t[1][0]) 
                       + b[2] * (t[2] - t[2][0]) + c for b, t, c in zip(weights, parent_roots, child_roots)])

In [247]:
strands_start = strands[::,1::]
strands_end = strands[::,:-1:]

childStrands_start = childStrands[:,1::]
childStrands_end = childStrands[:,:-1:]

# render the trimesh with meshplot
p = mp.plot(mesh.vertices, mesh.faces)
p.add_points(child_roots, c='red', shading=dict(point_size=.2))

# p.add_points(strands[:,0],                    c='green', shading=dict(point_size=.05))
# p.add_points(np.concatenate(np.concatenate(parent_roots)), c='blue',  shading=dict(point_size=.2))
# p.add_points(np.concatenate(childStrands),    c='red',   shading=dict(point_size=.2))

# p.add_lines(np.concatenate(strands_start), 
#             np.concatenate(strands_end), 
#             shading=dict(line_width=.005, line_color='green'))

p.add_lines(np.concatenate(childStrands_start),
            np.concatenate(childStrands_end),
            shading=dict(line_width=.005, line_color='red'))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.24…

2